In [1]:
import pandas as pd
from taxbrain import TaxBrain, differences_plot, distribution_plot

In [2]:
puf = pd.read_csv("~/puf.csv")

In [3]:
# Use the Medicare payroll tax since uncapped.
FICA_MC_BASE = 0.029
START_YEAR = 2021
END_YEAR = 2030
UBI = 6000
UBI_REFORM = {"UBI_21": UBI, "UBI_1820": UBI, "UBI_ecrt": 1}
reform_1pct = UBI_REFORM.copy()
reform_1pct["FICA_mc_trt"] = FICA_MC_BASE + 0.01
tb_1pct = TaxBrain(START_YEAR, END_YEAR, microdata=puf, reform=reform_1pct)
tb_1pct.run()

In [4]:
def window_total(tb, col):
    # Total over the full window.
    return tb.weighted_totals(col).loc["Difference"].sum()

ubi_cost = window_total(tb_1pct, "ubi")
ubi_cost / 1e12

16.3931776992

In [5]:
rev_per_1pct = window_total(tb_1pct, "combined")
rev_per_1pct / 1e12

1.106373169259087

In [6]:
needed_payroll_tax = (ubi_cost / rev_per_1pct).round(1)
needed_payroll_tax

14.8

In [7]:
# Budget-neutral
reform_bn_static = UBI_REFORM.copy()
reform_bn_static["FICA_mc_trt"] = FICA_MC_BASE + (needed_payroll_tax / 100)
tb_bn_static = TaxBrain(START_YEAR, END_YEAR, microdata=puf, reform=reform_bn_static)
tb_bn_static.run()

In [8]:
# Should be close to 1.
window_total(tb_bn_static, "combined") / ubi_cost

0.9959548890066374

In [9]:
# Budget-neutral with behavioral responses.
# Based on CBO (2012).
BEHAVIOR = {"sub": 0.25, "inc": -0.05}
reform_bn_beh = UBI_REFORM.copy()
reform_bn_beh["FICA_mc_trt"] = FICA_MC_BASE + 0.169
tb_bn_beh = TaxBrain(START_YEAR, END_YEAR, microdata=puf, reform=reform_bn_beh,
                     behavior=BEHAVIOR)
tb_bn_beh.run()

In [10]:
# Should be close to 1.
window_total(tb_bn_beh, "combined") / ubi_cost

0.9989242634968594